In [115]:
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [116]:
df = pd.read_csv('data.csv')

In [117]:
# Get completed games
comp_games = df[df['score_home'].notna()]
# Get uncompleted games
uncomp_games = df[df['score_home'].isna()]

In [118]:
# Determine winner and loser for completed games
for index, row in comp_games.iterrows():
    if comp_games.loc[index, 'score_home'] > comp_games.loc[index, 'score_away']:
        comp_games.loc[index, 'Home_Winner'] = 1
    else:
        comp_games.loc[index, 'Home_Winner'] = 0
    # else:
    #     comp_games.at[index, 'Winner'] = "T"

/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_21831/3030477943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_games.loc[index, 'Home_Winner'] = 0


In [119]:
# Remove scores (no cheating)
comp_games = comp_games.drop(['score_home', 'score_away'], axis=1)
uncomp_games = uncomp_games.drop(['score_home', 'score_away'], axis=1)

In [120]:
# Display training data
comp_games

,schedule_week,schedule_playoff,team_home,team_away,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,off_red_per,W_away,L_away,T_away,W-L%_away,PF_away,PA_away,PD_away,MoV_away,SoS_away,SRS_away,OSRS_away,DSRS_away,off_red_per_away,Home_Winner
0,1,False,Los Angeles Rams,Buffalo Bills,3,5,0,0.375,131,173,-42,-5.3,1.0,-4.3,-4.0,-0.3,0.5000,6,2,0,0.750,220,118,102,12.8,1.6,14.3,6.2,8.2,0.5357,0.0
1,1,False,Arizona Cardinals,Kansas City Chiefs,3,6,0,0.333,203,241,-38,-4.2,0.1,-4.1,-1.0,-3.1,0.5769,6,2,0,0.750,243,189,54,6.8,-0.6,6.1,9.3,-3.2,0.7353,0.0
2,1,False,Atlanta Falcons,New Orleans Saints,4,6,0,0.400,232,250,-18,-1.8,-1.4,-3.2,0.4,-3.6,0.6207,3,6,0,0.333,212,227,-15,-1.7,-0.6,-2.3,0.7,-3.0,0.5714,0.0
3,1,False,Carolina Panthers,Cleveland Browns,3,7,0,0.300,204,243,-39,-3.9,-1.2,-5.1,-1.9,-3.2,0.5455,3,5,0,0.375,200,199,1,0.1,0.2,0.3,3.0,-2.7,0.6129,0.0
4,1,False,Chicago Bears,San Francisco 49ers,3,6,0,0.333,187,216,-29,-3.2,0.7,-2.5,0.7,-3.2,0.5517,4,4,0,0.500,176,147,29,3.6,-1.7,2.0,-1.2,3.2,0.5769,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,8,False,New Orleans Saints,Las Vegas Raiders,3,6,0,0.333,212,227,-15,-1.7,-0.6,-2.3,0.7,-3.0,0.5714,2,6,0,0.250,183,201,-18,-2.3,-1.9,-4.2,-0.1,-4.0,0.5313,1.0
119,8,False,New York Jets,New England Patriots,6,3,0,0.667,196,176,20,2.2,2.6,4.8,1.8,2.9,0.6000,5,4,0,0.556,203,166,37,4.1,-0.2,3.9,-0.1,4.0,0.5000,0.0
120,8,False,Philadelphia Eagles,Pittsburgh Steelers,8,0,0,1.000,225,135,90,11.3,-2.0,9.3,5.5,3.8,0.7000,2,6,0,0.250,120,197,-77,-9.6,5.4,-4.2,-3.8,-0.3,0.4762,1.0
121,8,False,Seattle Seahawks,New York Giants,6,3,0,0.667,241,220,21,2.3,-2.3,0.0,3.6,-3.6,0.4828,6,2,0,0.750,163,157,6,0.8,0.2,1.0,-1.8,2.8,0.4615,1.0


In [121]:
# Set X and Y
X = comp_games.drop(['Home_Winner'], 1)
X_pred = uncomp_games
y = comp_games['Home_Winner']

# Scale data for more accurate predictions
from sklearn.preprocessing import scale
cols = [comp_games.columns[4:-1]]
for col in cols:
    X[col] = scale(X[col])
cols2 = [uncomp_games.columns[4:]]
for col in cols2:
    X_pred[col] = scale(X_pred[col])

/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_21831/57457822.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = comp_games.drop(['Home_Winner'], 1)


In [122]:
# Columnize any strings in dataset
def preprocess_features(X):
    output = pd.DataFrame(index = X.index)
    for col, col_data in X.items():
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
        output = output.join(col_data)
    return output

X = preprocess_features(X)
X_pred = preprocess_features(X_pred)

In [123]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [124]:
# Initialize algorithms
LR_clf = LogisticRegression()
SVC_clf = SVC()
xgb_clf = xgb.XGBClassifier(seed = 1)

# Fit algorithms to training data
LR_clf.fit(X_train, y_train)
SVC_clf.fit(X_train, y_train)
xgb_clf.fit(X_train, y_train)

# Predict using algorithms
LR_pred = LR_clf.predict(X_test)
SVC_pred = SVC_clf.predict(X_test)
XGB_pred = xgb_clf.predict(X_test)

# Cross validate algorithms
LR_score = cross_val_score(LR_clf, X, y)
SVC_score = cross_val_score(SVC_clf, X, y)
XGB_score = cross_val_score(xgb_clf, X, y)

In [125]:
# Print accuracy and score of algorithms
print(f'Logistic Regression:', '\nAccuracy:', accuracy_score(y_test, LR_pred), '\nF1 Score: ', f1_score(y_test, LR_pred), '\nCross Validation Score:', LR_score.mean())
print()
print(f'SVC:', '\nAccuracy:', accuracy_score(y_test, SVC_pred), '\nF1 Score: ', f1_score(y_test, SVC_pred), '\nCross Validation Score: ', SVC_score.mean())
print()
print(f'XGBoost:', '\nAccuracy:', accuracy_score(y_test, XGB_pred), '\nF1 Score: ', f1_score(y_test, XGB_pred), '\nCross Validation Score: ', XGB_score.mean())

Logistic Regression: 
Accuracy: 0.6923076923076923 
F1 Score:  0.7142857142857143 
Cross Validation Score: 0.5616666666666668

SVC: 
Accuracy: 0.7692307692307693 
F1 Score:  0.7999999999999999 
Cross Validation Score:  0.6916666666666667

XGBoost: 
Accuracy: 0.7692307692307693 
F1 Score:  0.7999999999999999 
Cross Validation Score:  0.5286666666666667


In [126]:
predictions = SVC_clf.predict(X_pred)

uncomp_games['Home_Winner'] = predictions

In [127]:
# Show predictions for week 9
pd.set_option('display.max_columns', None)
uncomp_games.groupby("schedule_week").get_group(10)[['team_home', 'team_away', 'Home_Winner']]

,team_home,team_away,Home_Winner
136,Carolina Panthers,Atlanta Falcons,1.0
137,Buffalo Bills,Minnesota Vikings,1.0
138,Chicago Bears,Detroit Lions,1.0
139,Green Bay Packers,Dallas Cowboys,0.0
140,Kansas City Chiefs,Jacksonville Jaguars,1.0
141,Las Vegas Raiders,Indianapolis Colts,1.0
142,Los Angeles Rams,Arizona Cardinals,1.0
143,Miami Dolphins,Cleveland Browns,1.0
144,New York Giants,Houston Texans,1.0
145,Pittsburgh Steelers,New Orleans Saints,1.0
